In [1]:
import pandas as pd
from thefuzz import fuzz

import warnings
warnings.filterwarnings("ignore")

## Readin and Initial Overview of Data

In [2]:
df = pd.read_excel('evictions_2023.xlsx')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16657 entries, 0 to 16656
Data columns (total 25 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   JP PRECINCT            16657 non-null  object        
 1   CaseNumber             16657 non-null  object        
 2   CaseFileDate           16657 non-null  datetime64[ns]
 3   CORP                   16657 non-null  object        
 4   Plaintiff              16657 non-null  object        
 5   PERSON_ALIAS_ID        16657 non-null  int64         
 6   PlaintiffPhone         16657 non-null  object        
 7   PlaintiffAddress       16657 non-null  object        
 8   Pl_City                16657 non-null  object        
 9   Pl_State               16657 non-null  object        
 10  Pl_Zip                 16657 non-null  object        
 11  DefendantAddress       16028 non-null  object        
 12  Unit Number            8116 non-null   object        
 13  C

In [4]:
og_data = df.shape[0]
print(f'Original Dataframe Length: {og_data}')

Original Dataframe Length: 16657


## Get and Rename Relevant Columns

In [5]:
for col in df.columns:
    
    print(col)

JP PRECINCT
CaseNumber
CaseFileDate
CORP
Plaintiff
PERSON_ALIAS_ID
PlaintiffPhone
PlaintiffAddress
Pl_City
Pl_State
Pl_Zip
DefendantAddress
Unit Number
CITY_NAME
STATE_CD
POSTAL_CD
Defendant
Def_Phone
CaseStatusDescription
JUDGMENT_DT
Judgment
Disposition
DispositionDate
DISP_AMT
CASE_TYP_DESC


In [6]:
# get relevant columns and rename for clarity
df = df[['CaseNumber',        # used to distinguish unique cases
         'CaseFileDate',      # used for case date
         'JUDGMENT_DT',       # used for date case was ruled on
         'DispositionDate',   # used for date of disposition
         'CITY_NAME',         # used for city location
         'POSTAL_CD',         # used for zip code
         'Disposition',        # used to determine if case was dismissed
         ]]   

df = df.rename(columns={'CaseNumber'      : 'case_number',
                        'CaseFileDate'    : 'file_date',
                        'JUDGMENT_DT'     : 'judgement_date',
                        'DispositionDate' : 'disposition_date',
                        'CITY_NAME'       : 'city',
                        'POSTAL_CD'       : 'zip_code',
                        'Disposition'     : 'disposition'})

df.head(15)

,case_number,file_date,judgement_date,disposition_date,city,zip_code,disposition
0,11E2102330,2021-12-13,2024-09-17,2024-09-17,SAN ANTONIO,78221,Judgment for Plaintiff (OCA)
1,11E2301639,2023-05-10,2024-05-01,2024-05-01,SAN ANTONIO,78227,Agreed Judgments (OCA)
2,11E2303045,2023-08-16,2024-01-11,2023-12-08,San Antonio,78214,Dismissed
3,11E2303045,2023-08-16,2024-01-11,2023-12-08,San Antonio,78214,Dismissed
4,11E2303297,2023-08-30,2024-02-08,2024-02-08,San Antonio,78251,Default Judgments (OCA)
5,11E2303925,2023-10-09,2024-01-11,2024-01-11,San Antonio,78253-4301,Judgment for Plaintiff (OCA)
6,11E2304048,2023-10-17,2024-01-04,2024-01-04,San Antonio,78226,Judgment for Plaintiff (OCA)
7,11E2304072,2023-10-18,2024-07-09,2024-07-09,SAN ANTONIO,78221,Take Nothing Judgment (OCA)
8,11E2304114,2023-10-19,2024-01-11,2024-01-11,SAN ANTONIO,78221,Judgment for Plaintiff (OCA)
9,11E2304258,2023-10-26,2024-06-18,2024-06-18,SAN ANTONIO,78214,Judgment for Plaintiff (OCA)


## Getting Relevant City Values
* Used the fuzz to adjust for misspellings and abreviations of San Antonio
* Data includes surrounding areas of San Antonio it is unclear if it shold be included

In [7]:
# adjust city values for misspellings of san antonio, lowercase all values, and keep only San Antonio data
def fuzzy_sa(value):
    ''' takes in a pandas value and returns san antonio if string is a 90 or higher ratio to san antonio
        otherwise returns lower cased version of the original string'''
    
    value = str(value).lower()
    
    sa1 = 'san antonio'
    sa2 = 'sa'
    
    ratio1 = fuzz.ratio(sa1,value)
    ratio2 = fuzz.ratio(sa2,value)
    
    if (ratio1 >= 80) or (ratio2 >= 60):
        
        return 'san antonio'
    
    else:
        
        return value

df['city'] = df.city.apply(fuzzy_sa)

# remove cities not in list of acceptaple cities
cities = ['san antonio']

df = df[df.city.isin(cities)]

df.head(15)

,case_number,file_date,judgement_date,disposition_date,city,zip_code,disposition
0,11E2102330,2021-12-13,2024-09-17,2024-09-17,san antonio,78221,Judgment for Plaintiff (OCA)
1,11E2301639,2023-05-10,2024-05-01,2024-05-01,san antonio,78227,Agreed Judgments (OCA)
2,11E2303045,2023-08-16,2024-01-11,2023-12-08,san antonio,78214,Dismissed
3,11E2303045,2023-08-16,2024-01-11,2023-12-08,san antonio,78214,Dismissed
4,11E2303297,2023-08-30,2024-02-08,2024-02-08,san antonio,78251,Default Judgments (OCA)
5,11E2303925,2023-10-09,2024-01-11,2024-01-11,san antonio,78253-4301,Judgment for Plaintiff (OCA)
6,11E2304048,2023-10-17,2024-01-04,2024-01-04,san antonio,78226,Judgment for Plaintiff (OCA)
7,11E2304072,2023-10-18,2024-07-09,2024-07-09,san antonio,78221,Take Nothing Judgment (OCA)
8,11E2304114,2023-10-19,2024-01-11,2024-01-11,san antonio,78221,Judgment for Plaintiff (OCA)
9,11E2304258,2023-10-26,2024-06-18,2024-06-18,san antonio,78214,Judgment for Plaintiff (OCA)


## Normalizing Data in zip_code to 5 Digit Values 
* Data in some values is presented in 9 digit format while the majority is in 5 digit format

In [8]:
# get first five digits of values in zip_code column

df['zip_code'] = df.zip_code.apply(lambda x : str(x)[:5])

df.head(15)

,case_number,file_date,judgement_date,disposition_date,city,zip_code,disposition
0,11E2102330,2021-12-13,2024-09-17,2024-09-17,san antonio,78221,Judgment for Plaintiff (OCA)
1,11E2301639,2023-05-10,2024-05-01,2024-05-01,san antonio,78227,Agreed Judgments (OCA)
2,11E2303045,2023-08-16,2024-01-11,2023-12-08,san antonio,78214,Dismissed
3,11E2303045,2023-08-16,2024-01-11,2023-12-08,san antonio,78214,Dismissed
4,11E2303297,2023-08-30,2024-02-08,2024-02-08,san antonio,78251,Default Judgments (OCA)
5,11E2303925,2023-10-09,2024-01-11,2024-01-11,san antonio,78253,Judgment for Plaintiff (OCA)
6,11E2304048,2023-10-17,2024-01-04,2024-01-04,san antonio,78226,Judgment for Plaintiff (OCA)
7,11E2304072,2023-10-18,2024-07-09,2024-07-09,san antonio,78221,Take Nothing Judgment (OCA)
8,11E2304114,2023-10-19,2024-01-11,2024-01-11,san antonio,78221,Judgment for Plaintiff (OCA)
9,11E2304258,2023-10-26,2024-06-18,2024-06-18,san antonio,78214,Judgment for Plaintiff (OCA)


## After Dropping Duplicate rows Duplicated Case Numbers Still Exist Due to Differing Dispositions

In [9]:
df = df.drop_duplicates().reset_index(drop=True)

df[df.case_number == '41E2406496']

,case_number,file_date,judgement_date,disposition_date,city,zip_code,disposition
14850,41E2406496,2024-10-24,2024-11-19,2024-11-19,san antonio,78219,Default Judgments (OCA)
14851,41E2406496,2024-10-24,2024-11-19,2024-11-08,san antonio,78219,Dismissed for Want of Prosecution (OCA)


## Examening Dismissed Cases

In [10]:
# add boolean column to identify rows with dismissed in disposition columns
df['dismissed'] = df.disposition.apply(lambda x : 'Dismissed' in x)

# get list of case numbers that were dismissed
dis_df = df[df.dismissed == True]

dismissed = list(set(dis_df.case_number.tolist()))

### Non-suited or Dismissed by Plaintiff  and Judgment for Plaintiff Appear on the Same Date
* File and judgement dates are the same across case numbers
* Disposition date changes with disposition

In [11]:
# example 1
df_case = df[df.case_number == dismissed[0]]
df_case

,case_number,file_date,judgement_date,disposition_date,city,zip_code,disposition,dismissed
1800,11E2402950,2024-07-10,2024-07-25,2024-07-25,san antonio,78251,Non-suited or Dismissed by Plaintiff (OCA),True


In [12]:
# example 2
df_case = df[df.case_number == dismissed[11]]
df_case

,case_number,file_date,judgement_date,disposition_date,city,zip_code,disposition,dismissed
13062,41E2402204,2024-04-16,2024-05-28,2024-05-28,san antonio,78213,Default Judgments (OCA),False
13063,41E2402204,2024-04-16,2024-05-28,2024-05-14,san antonio,78213,Dismissed for Want of Prosecution (OCA),True


In [13]:
# example 3
df_case = df[df.case_number == dismissed[50]]
df_case

,case_number,file_date,judgement_date,disposition_date,city,zip_code,disposition,dismissed
11254,31E2404123,2024-10-03,2024-11-14,2024-12-06,san antonio,78255,Appealed,False
11255,31E2404123,2024-10-03,2024-11-14,2024-11-14,san antonio,78255,Default Judgments (OCA),False
11256,31E2404123,2024-10-03,2024-11-14,2024-10-24,san antonio,78255,Dismissed for Want of Prosecution (OCA),True


## Eleminating Duplicate Case Numbers
* Keeping row with oldest disposition date for each case number 
* All duplicate case numbers dropped successfully

In [14]:
df = df.sort_values(by='disposition_date',ascending=False).reset_index(drop=True)

df = df.drop_duplicates(subset='case_number', keep='first').reset_index(drop=True)

df.head(10)

,case_number,file_date,judgement_date,disposition_date,city,zip_code,disposition,dismissed
0,31E2405002,2024-11-26,2024-12-17,2025-01-10,san antonio,78232,Appealed,False
1,31E2405011,2024-12-02,2024-12-19,2025-01-10,san antonio,78257,Appealed,False
2,31E2404980,2024-11-25,2024-12-17,2025-01-10,san antonio,78247,Appealed,False
3,31E2404897,2024-11-19,2024-12-10,2025-01-07,san antonio,78229,Appealed,False
4,31E2404217,2024-10-10,2024-10-31,2025-01-06,san antonio,78216,Appealed,False
5,41E2400487,2024-01-18,2024-02-23,2025-01-03,san antonio,78233,Default Judgments (OCA),False
6,41E2401335,2024-02-20,2024-05-15,2025-01-03,san antonio,78222,Judgment for Plaintiff (OCA),False
7,41E2401557,2024-03-11,2024-04-23,2025-01-03,san antonio,78220,Default Judgments (OCA),False
8,31E2404641,2024-11-07,2024-12-12,2024-12-27,san antonio,78213,Appealed,False
9,31E2404660,2024-11-08,2024-12-11,2024-12-27,san antonio,78232,Appealed,False


# Dropping Cases not Likely to Result in Eviction
* **Likely Eviction**
    * Default Judgments (OCA)
    * Judgment for Plaintiff (OCA)
* **Ambiguous or Likely No Eviction**
    * Appealed
    * Take Nothing Judgment (OCA)
    * Non-suited or Dismissed by Plaintiff (OCA)
    * Agreed Judgments (OCA)
    * Dismissed for Want of Prosecution (OCA)
    * Dismissed
    * Trial/Hearing by Judge/Hearing Officer (OCA)
    * Dismissed for Want of Jurisdiction (OCA)
    
    
    
    
    

In [19]:
# removing rows where disposition is not likely to result in eviction
evict = ['Default Judgments (OCA)',
         'Judgment for Plaintiff (OCA)']

df = df[df.disposition.isin(evict)]

df.head(10)

,case_number,file_date,judgement_date,disposition_date,city,zip_code,disposition,dismissed
5,41E2400487,2024-01-18,2024-02-23,2025-01-03,san antonio,78233,Default Judgments (OCA),False
6,41E2401335,2024-02-20,2024-05-15,2025-01-03,san antonio,78222,Judgment for Plaintiff (OCA),False
7,41E2401557,2024-03-11,2024-04-23,2025-01-03,san antonio,78220,Default Judgments (OCA),False
26,31E2404952,2024-11-21,2024-12-19,2024-12-19,san antonio,78230,Default Judgments (OCA),False
27,21E2407650,2024-12-09,2024-12-19,2024-12-19,san antonio,78251,Default Judgments (OCA),False
28,21E2407658,2024-12-09,2024-12-19,2024-12-19,san antonio,78229,Judgment for Plaintiff (OCA),False
29,21E2407657,2024-12-09,2024-12-19,2024-12-19,san antonio,78229,Default Judgments (OCA),False
30,21E2407656,2024-12-09,2024-12-19,2024-12-19,san antonio,78229,Default Judgments (OCA),False
31,21E2407655,2024-12-09,2024-12-19,2024-12-19,san antonio,78229,Default Judgments (OCA),False
32,21E2407654,2024-12-09,2024-12-19,2024-12-19,san antonio,78229,Default Judgments (OCA),False


## Dropping Filter Columns

In [20]:
df = df[['case_number',
         'disposition_date',
         'zip_code']]

# Full Prep

In [35]:
def get_prepared_eviction_data():
    '''Prepare eviction data for project'''
    
    # read in unprepared data
    df = pd.read_excel('evictions_2023.xlsx')
    
    # get relevant columns and rename for clarity
    df = df[['CaseNumber',        # used to distinguish unique cases
             'CaseFileDate',      # used for case date
             'JUDGMENT_DT',       # used for date case was ruled on
             'DispositionDate',   # used for date of disposition
             'CITY_NAME',         # used for city location
             'POSTAL_CD',         # used for zip code
             'Disposition',        # used to determine if case was dismissed
             ]]   

    df = df.rename(columns={'CaseNumber'      : 'case_number',
                            'CaseFileDate'    : 'file_date',
                            'JUDGMENT_DT'     : 'judgement_date',
                            'DispositionDate' : 'disposition_date',
                            'CITY_NAME'       : 'city',
                            'POSTAL_CD'       : 'zip_code',
                            'Disposition'     : 'disposition'})
    
    # adjust values in cities for misspellings of san antonio
    df['city'] = df.city.apply(fuzzy_sa)

    # remove cities not in list of acceptaple cities
    cities = ['san antonio']

    df = df[df.city.isin(cities)]

    # get first five digits of values in zip_code column
    df['zip_code'] = df.zip_code.apply(lambda x : str(x)[:5])

    # remove duplicate case numbers keeping the latest according to the disposition date
    df = df.sort_values(by='disposition_date',ascending=False).reset_index(drop=True)

    df = df.drop_duplicates(subset='case_number', keep='first').reset_index(drop=True)
    
    # removing rows where disposition is not likely to result in eviction
    evict = ['Default Judgments (OCA)',
             'Judgment for Plaintiff (OCA)']

    df = df[df.disposition.isin(evict)]
    
    # remove columns used for filtering
    df = df[['case_number',
             'disposition_date',
             'zip_code']].reset_index(drop=True)
    
    df.to_excel('evictions_prepared.xlsx')
    
    return df
    
    
def fuzzy_sa(value):
    ''' takes in a pandas value and returns san antonio if string is a 90 or higher ratio to san antonio
        otherwise returns lower cased version of the original string '''
    
    value = str(value).lower()
    
    sa1 = 'san antonio'
    sa2 = 'sa'
    
    ratio1 = fuzz.ratio(sa1,value)
    ratio2 = fuzz.ratio(sa2,value)
    
    if (ratio1 >= 80) or (ratio2 >= 60):
        
        return 'san antonio'
    
    else:
        
        return value
    

In [36]:
df = get_prepared_eviction_data()

In [37]:
df

,case_number,disposition_date,zip_code
0,41E2400487,2025-01-03,78233
1,41E2401335,2025-01-03,78222
2,41E2401557,2025-01-03,78220
3,31E2404952,2024-12-19,78230
4,21E2407650,2024-12-19,78251
...,...,...,...
13800,31E2304832,2024-01-02,78256
13801,31E2304823,2024-01-02,78249
13802,31E2304825,2024-01-02,78249
13803,31E2304811,2024-01-02,78249
